<a href="https://colab.research.google.com/github/AG211695/LLM_Langgraph_Chatbot/blob/main/ChatBot_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langgraph langsmith

In [2]:
!pip install langchain langchain_groq langchain_community

In [3]:
from google.colab import userdata
groq_api_key = userdata.get('groq_api_key')
langsmith = userdata.get('langsmith_api_key')

In [4]:
import os
os.environ["LANGCHAIN_API_KEY"]= langsmith
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="ChatBotLanggraph"

In [5]:
from langchain_groq import ChatGroq

In [6]:
llm_model=ChatGroq(groq_api_key=groq_api_key,model_name='llama-3.1-70b-versatile')
llm_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7e9a33ce9fc0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7e9a33ce92d0>, model_name='llama-3.1-70b-versatile', groq_api_key=SecretStr('**********'))

**So far all the necessary connections and imports have been made. Now, using Langgraph, building of ChatbBot Begins**

In [7]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages

In [8]:
class State(TypedDict):
  messages:Annotated[list,add_messages]

graph_builder=StateGraph(State)

In [9]:
graph_builder

In [10]:
# Creating a function to make the chatbot to interact with LLM
# In this case it's going to interact with latest Llama 3

def chatbot(state:State):
  return {"messages":llm_model.invoke(state['messages'])}

In [11]:
graph_builder.add_node("chatbot",chatbot)

In [12]:
graph_builder

In [13]:
#Connecting nodes to chatbot and creating start -- Chatbot -- end flow
graph_builder.add_edge(START,'chatbot')
graph_builder.add_edge('chatbot',END)

In [14]:
#Compiling graph_builder
graph=graph_builder.compile()

In [15]:
#Displaying the flow
from IPython.display import Image, display
try:
  display(Image(graph.get_graph().draw_mermaid.png()))
except Exception:
  pass

In [18]:
#Executing, using the chatbot
while True:
  user_input=input("User: ")
  if user_input.lower() in ['quit','q']:
    print('Good Bye!')
    break
  for event in graph.stream({'messages':("user",user_input)}):
    print(event.values())
    for value in event.values():
      print(value['messages'])
      print('Assistant:',value['messages'].content)

User: hi
dict_values([{'messages': AIMessage(content='How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 36, 'total_tokens': 44, 'completion_time': 0.032, 'prompt_time': 0.009145294, 'queue_time': 0.005426725, 'total_time': 0.041145294}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None}, id='run-0079e3ae-7a5e-4be4-9f78-a71dbcd91b3b-0', usage_metadata={'input_tokens': 36, 'output_tokens': 8, 'total_tokens': 44})}])
content='How can I assist you today?' response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 36, 'total_tokens': 44, 'completion_time': 0.032, 'prompt_time': 0.009145294, 'queue_time': 0.005426725, 'total_time': 0.041145294}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None} id='run-0079e3ae-7a5e-4be4-9f78-a71dbcd91b3b-0' usage_metadata={'input_tokens': 36, 